In [1]:
%run StdPackages.ipynb
d['rawData'] = os.path.join(d['data'],'rawData69') # add
d['processedData'] = os.path.join(d['data'],'processedData') # update to raw data folder
os.chdir(d['py'])

No clean-up of work-folder


# Emission trends 

We take emissions data using IO69 codes and look at how intensities have evolved between 1990 and some year $t$. We then define average growth rates and map them to other useful dimensions (e.g. 146 green reform codes).

Load data:

In [2]:
db = GpyDB(os.path.join(d['processedData'], 'IO69'))
auxMaps = DbFromExcel.map(DbFromExcel.simpleLoad(os.path.join(d['data'], 'GR2019_mappings.xlsx'))['AuxMaps'])
ws = db.ws
t0 = db('t')[0] 

For production sectors, define value of output from summing over inputs:

In [3]:
valueOfOutput = adj.rc_pd(db('vD'), db('s_p')).groupby(['t','s']).sum()

Define CO2 intensity over time:

In [4]:
μ = db('qCO2')/valueOfOutput

Define average growth rates in intensity since 1990:

In [5]:
g = ((μ/μ.xs(t0)).pow(1/(pd.Series(μ.index.get_level_values('t'), index = μ.index)-t0))-1).fillna(0)

Map to 146 levels:

In [6]:
m = auxMaps['s69tos146'].vals
m = m.set_levels([l.astype(str) for l in m.levels]) # str definition of sectors
μ146 = adjMultiIndex.applyMult(μ, m).droplevel('s').rename_axis(['t','s'])
g146 = adjMultiIndex.applyMult(g, m).droplevel('s').rename_axis(['t','s'])

Map to GR levels:

In [7]:
m = auxMaps['s146tosGR'].vals
m = m.set_levels([l.astype(str) for l in m.levels]) # str definition of sectors
μGR = adjMultiIndex.applyMult(μ146, m).droplevel('s').rename_axis(['t','s'])
gGR = adjMultiIndex.applyMult(g146, m).droplevel('s').rename_axis(['t','s'])

Store data:

In [8]:
dbOut = GpyDB(name = 'emissionTrends', data_folder = d['processedData'], ws = ws)
dbOut['uCO2_s69'] = μ
dbOut['uCO2_s146'] = μ146
dbOut['uCO2_sGR'] = μGR
dbOut['gCO2_s69'] = g
dbOut['gCO2_s146'] = g146
dbOut['gCO2_sGR'] = gGR
dbOut.export()